In [1]:
import gym
env = gym.make("Taxi-v2").env
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [2]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


In [3]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [4]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [5]:
import numpy as np
import time
import pickle, os
import random

num_episodes = 10000
max_steps_per_episode = 2000

learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

In [6]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))

In [7]:
rewards_all_episodes = []

# Q-learning Algorithm
for episode in range(num_episodes):
    state = env.reset()
    
    done = False
    rewards_current_episode = 0
    
    for step in range(max_steps_per_episode):
        
        #Exploration-Exploitation tradeoff
        exploration_rate_threshold = random.uniform(0,1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:])
        else:
            action = env.action_space.sample()
            
        new_state, reward, done, info = env.step(action)
        
        #Update Q-table for Q(s,a)
        q_table[state, action] = q_table[state, action] * (1- learning_rate) + learning_rate * (reward + discount_rate * np.max(q_table[new_state,:]))
        
        state = new_state
        rewards_current_episode += reward
        
        if done == True:
            break
        
        
    # Exploration rate decay
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate * episode)
        
    rewards_all_episodes.append(rewards_current_episode)

In [8]:
# Calculate and print the avg reward per thousand episode
reward_per_thousand_episodes = np.split(np.array(rewards_all_episodes), num_episodes/1000)
count = 1000

print("********Average reward per thousand episodes********\n")
for r in reward_per_thousand_episodes:
    print(count, " : ", str(sum(r/1000)))
    count += 1000

********Average reward per thousand episodes********

1000  :  -579.9719999999988
2000  :  -8.139999999999999
3000  :  3.526999999999987
4000  :  6.4749999999999766
5000  :  7.431999999999964
6000  :  7.886999999999964
7000  :  7.9609999999999665
8000  :  7.996999999999976
9000  :  7.9039999999999715
10000  :  8.106999999999967


In [9]:
# Print updated Q-table
print("\n\n********Q-table********\n")
print(q_table)



********Q-table********

[[  0.           0.           0.           0.           0.
    0.        ]
 [  5.48940992   4.54792966   4.90395154   6.47199792  11.84784175
   -0.85758731]
 [ 10.68515198  10.90166585   9.7206351   11.36881377  14.11880599
    2.7343633 ]
 ...
 [ 13.59532359  15.2715212   13.26889798   9.74971714   4.01274282
    4.39952649]
 [ -4.28491831  -3.23915183  -3.7738       8.65918882 -11.83702349
  -11.89157   ]
 [ 15.98899278  12.83247148  14.5502086   18.79950672   5.88330775
    7.6176738 ]]


In [10]:

from IPython.display import clear_output
from time import sleep

In [12]:
for episode in range(5):
    step = env.reset()
    done = False
    print("*****EPISODE ", episode+1, "*****\n\n\n\n")
    time.sleep(1)
    
    for step in range(max_steps_per_episode):  
        clear_output(wait=True)
        env.render()
        time.sleep(0.5)
        
        action = np.argmax(q_table[state,:])        
        new_state, reward, done, info = env.step(action)
        
        if done:
            clear_output(wait=True)
            env.render()
            if reward == 20:
                print("****taxi reached the destination!****")
                time.sleep(3)
            else:
                print("****taxi didn't reached the destination!****")
                time.sleep(3)
            clear_output(wait=True)
            break
            
        state = new_state    
        
        env.close()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
****taxi reached the destination!****
